<font size="18">Chain Gang on a Fence</font>

In [ ]:
# set this notebook to use a large part of the browser window width
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:80% !important; }</style>"))

# Introduction

We consider the setup where N people are holding hands in a chain, so in series and the first person
puts one hand on an electric fence.

Given normal human resistances, we intend here to calculate the currents through arms and bodies of each person.

This is mainly to refute a false belief 
stated on [this Quora webpage](https://www.quora.com/If-a-line-of-people-hold-hands-and-the-person-at-the-end-touches-an-electric-fence-who-will-get-the-greatest-shock)
that somehow, the last person in the chain would receives the most current through his arms or body.
This belief makes N-1 people play some pranks on the Nth person, and they are wrongly convinced that the Nth person suffers the most. It must be group psychology, tricking the N-1 people into that.


# Model Setup

The network of resistances can be represented as in the below figure.

In [ ]:
from IPython.display import Image
Image(filename='ChainGangOnAFence.png', width=1000, height=3000)

Using Kirchhoff current and voltege laws we get the equations.

In [ ]:
Image(filename='KirchhoffCurrentLawUpperNodes.tex.png')

In [ ]:
Image(filename='KirchhoffCurrentLawLowerNodes.tex.png')

In [ ]:
Image(filename='KirchhoffVoltageLawForAllUpperLines.tex.png')

In [ ]:
Image(filename='KirchhoffVoltageLawForAllUpDownLines.tex.png')

In [ ]:
Image(filename='KirchhoffVoltageLawForAllLowerLines.tex.png')

This gives a system of 5N linear equations. 
for 3N current variables and 2(N+1) voltage variables, so for 5N+2 variables.
However, in addition, we set v(l,0) to 0 and v(u,0) to the battery voltage, so we then have 5N+2 equations as well.
That is nicely and easily solvable using some linear algebra.

Even though, there is only one solution here and we do not have an objective, to solve this,
we can still use the Integer Linear Programming (ILP) solver Gurobi. There will be only continuous variables
and the problem is a feasibility rather than optimization problem.
We code this as follows.

In [ ]:
from gurobipy import *

# https://www.agrisellex.co.uk/blogs/blog/electric-fencing-in-dry-soil-conditions
# claims:
wet_ground_resistance_ohms = 50
dry_ground_resistance_ohms = 10000

def set_resistances(N, 
                    hand_to_hand_resistance_ohm = 10000, 
                    neck_to_feet_resistance_ohm = 10000, 
                    person_thru_ground_to_prev_person_resistance_ohm = wet_ground_resistance_ohms):    
    verbose = 0

    r_a = [hand_to_hand_resistance_ohm] * N
    r_b = [neck_to_feet_resistance_ohm] * N
    r_g = [person_thru_ground_to_prev_person_resistance_ohm] * N
    
    if verbose >= 1:
        print(r_a)

    return r_a, r_b, r_g

def print_resistances(r_a, r_b, r_g):
    for const_name, const_struct in list(zip(['r_a', 'r_b', 'r_g'], [r_a, r_b, r_g])):
        print(const_name + ' (Ohm) = ')
        print(const_struct)

def calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000):
    verbose = 0
    
    N = len(r_a)
    assert len(r_b) == N
    assert len(r_g) == N
    
    m = Model()

    rN = range(0,N)         # [0, 1, .. N-1]
    rNplus1 = range(0,N+1)  # [0, 1, .. N]

    rCurrents = list(rN)
    rVoltages = list(rNplus1)  # we also define v0 as a variable even if this battery voltage is fixed and known ahead.
    
    if verbose >= 1:
        print(rCurrents)
        print(rVoltages)

    # create all the 4N continuous variables:
    i_a = m.addVars(rCurrents, vtype=GRB.CONTINUOUS, name='i_a')  # upper node currents (going to the right thru arms)
    i_b = m.addVars(rCurrents, vtype=GRB.CONTINUOUS, name='i_b')  # body currents (flowing down from nack to legs)
    i_g = m.addVars(rCurrents, vtype=GRB.CONTINUOUS, name='i_g')  # lower node to battery anode (in ground) currents
    vu = m.addVars(rVoltages, vtype=GRB.CONTINUOUS, name='v')    # v0 (battery neck) and N voltages at person i neck for i in 1..N
    vd = m.addVars(rVoltages, vtype=GRB.CONTINUOUS, name='v')    # v0 (battery feet) and N voltages at person i feet for i in 1..N

    assert len(i_a) == N
    assert len(i_b) == N
    assert len(i_g) == N
    assert len(vu) == N+1
    assert len(vd) == N+1

    # (1) create all the 3N linear current constraints:
    for i in rCurrents:  # there are N of them: 0..N-1
        if i < N-1:
            m.addConstr(i_a[i  ] == i_a[i+1] + i_b[i  ], 'i_a_constr{:d}'.format(i))
            m.addConstr(i_g[i  ] == i_b[i  ] + i_g[i+1], 'i_b_constr{:d}'.format(i))
        else:
            m.addConstr(i_a[i  ] ==            i_b[i  ], 'i_a_constr{:d}'.format(i))
            m.addConstr(i_g[i  ] == i_b[i  ]           , 'i_b_constr{:d}'.format(i))

    # (2) create all the 2N linear current constraints:
    for i in rCurrents: # there are N+1 of them: 0,..N
        r = r_a[i] if i == 0 else r_a[i-1] + r_a[i]
        m.addConstr(vu[i  ] - r      * i_a[i] == vu[i+1], 'v_upper{:d}'.format(i))                 
        m.addConstr(vu[i+1] - r_b[i] * i_b[i] == vd[i+1], 'v_up_down{:d}'.format(i))
        m.addConstr(vd[i+1] - r_g[i] * i_g[i] == vd[i  ], 'v_lower{:d}'.format(i))

    # (3) create all the 2 linear voltage constraints, sespecially for the battery:
    # Set the battery voltage:
    m.addConstr(vu[0] == battery_voltage, 'vu_0_fix')  # 5000V is a typical average voltage for an average fence battery
    # See: https://hypertextbook.com/facts/2001/NicoleCastellano.shtml
    m.addConstr(vd[0] == 0, 'vl_0_fix')  # 0V is the typical voltage for the ground

    m.optimize()
    
    return m, i_a, i_b, i_g, vu, vd
        
def print_results(m, i_a, i_b, i_g, vu, vd):    
    for var_name, var_struct in list(zip(['i_a', 'i_b', 'i_g', 'vu', 'vd'], [i_a, i_b, i_g, vu, vd])):
        values = m.getAttr('x', var_struct)
        s = var_name
        if var_name in ['i_a', 'i_b', 'i_g']:
            s += ' (mA) = '
            vals = ['{:d}'.format(int(values[v] * 1000)) for v in values]
        else:
            assert var_name in ['vu', 'vd']
            s += ' (kV) = '
            vals = ['{:.2f}'.format((values[v] / 1000)) for v in values]
        print(s)
        print(vals)

# Calculation Examples

## Example with 2 Persons on Dry Ground

In [ ]:
N = 2  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, 
                                hand_to_hand_resistance_ohm = 10000, 
                                neck_to_feet_resistance_ohm = 5000, 
                                person_thru_ground_to_prev_person_resistance_ohm = dry_ground_resistance_ohms)
m, i_a, i_b, i_g, vu, vd = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, vu, vd)

So all current series i_a and i_b (and even i_g) show lower and lower values, when going from the first to the last person.
This means that in all respects, in arms and body, the Nth person is the one suffering the least, contrary to the popular false belief.

## Example with 3 Persons on Dry Ground

In [ ]:
N = 3  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, 
                                hand_to_hand_resistance_ohm = 10000, 
                                neck_to_feet_resistance_ohm = 5000,
                                person_thru_ground_to_prev_person_resistance_ohm = wet_ground_resistance_ohms)
m, i_a, i_b, i_g, vu, vd = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, vu, vd)

Again, in all respects, in arms and body, the Nth person is the one suffering the least, contrary to the popular false belief.

## Example with 4 Persons on Wet Ground

In [ ]:
N = 4  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, 
                                hand_to_hand_resistance_ohm = 10000, 
                                neck_to_feet_resistance_ohm = 5000,
                                person_thru_ground_to_prev_person_resistance_ohm = wet_ground_resistance_ohms)
m, i_a, i_b, i_g, vu, vd = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, vu, vd)

## Example with 4 Persons on Wet Ground, where Only the 4th is Barefoot in Wet Grass

We still assume wet grass here for all, but now assume even unrealistically low resistances for the last person's arm and body.

In [ ]:
N = 4  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, 
                                hand_to_hand_resistance_ohm = 10000, 
                                neck_to_feet_resistance_ohm = 5000,
                                person_thru_ground_to_prev_person_resistance_ohm = wet_ground_resistance_ohms)
r_a[N-1] = 50  # even almost zero arm resistance resulting from good wet contact with ground
r_b[N-1] = 50  # even almost zero body resistance resulting from good wet contact with ground
m, i_a, i_b, i_g, vu, vd = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, vu, vd)

So even if the resistance of the arm and body of the last person are supposed to be unrealistically low, he would not suffer great currents due to being protected by the resistance of all the people between him and the fence. This totally disproves the false believe that the last person would suffer the most. He suffers the least instead.

## Example with 4 Persons on Wet Ground, where Only the First and the Last are Barefoot in Wet Grass


In [ ]:
N = 4  # number of people in the chain gang
r_a, r_b, r_g = set_resistances(N, 
                                hand_to_hand_resistance_ohm = 10000, 
                                neck_to_feet_resistance_ohm = 5000,
                                person_thru_ground_to_prev_person_resistance_ohm = wet_ground_resistance_ohms)
r_a[0] = 50  # even almost zero arm resistance resulting from good wet contact with ground
r_b[0] = 50  # even almost zero body resistance resulting from good wet contact with ground

r_a[N-1] = 50  # even almost zero arm resistance resulting from good wet contact with ground
r_b[N-1] = 50  # even almost zero body resistance resulting from good wet contact with ground

m, i_a, i_b, i_g, vu, vd = calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000)
print_results(m, i_a, i_b, i_g, vu, vd)

Clearly the first person gets electrocuted but the last is still totally safe.

# A Better Way to Play the Prank

So the belief that the last person in the chain would somehow get the most current is clearly false.
Chaining up in series would be the [Bart SImpson](https://simpsons.fandom.com/wiki/Bart_Simpson) way to play the prank. Haahaa! :)

In [ ]:
Image(filename='BartSimpsonLaughingAnimated.gif')

But if you want to play a prank on the last person in the chain, trying to get more current through any body part than through the first N-1 persons and cannot force person N to wear more conductive shoes than the N-1 previous persons are wearing, there is a way...

It's putting the N-1 persons in parallel i.o. in series of course, as in the following figure.

In [ ]:
Image(filename='ParallelChainGangOnAFence.png', width=600, height=1000)

We could write a function calculate_currents_and_voltages(r_a, r_b, r_g, battery_voltage=5000) setting up the Kirchhoff laws, resulting in different equations here.
But it suffices to first understand that N-1 people shunted in parallel behave as a single person
of which the arm resistance is 1/(N-1) of a single person and of which the body resistance is also 1/(N-1) of a single person, as is done in the right halve of the figure above.
So the parallel N-1 people can be replaced by the equivalent network of 1 person with these 1/(N-1) lower resistances.

Now, the person N will roughly suffer N-1 times as much as the other N-1 people playing the prank.
This is clearly the way [Martin Prince Jr](https://simpsons.fandom.com/wiki/Martin_Prince) would do it! :)

In [ ]:
Image(filename='MartinPrinceJrLaughing.gif')

# Conclusion

So, let's agree that science should be taught before pranking age. :)

You can find [an article on Medium](https://logicallyyours.com/on-computation) refuting the same false belief but with as little physics / mathematical knowledge as possible. For more computational diversions from Logically Yours, have a look [at this blog](https://logicallyyours.com/on-computation). 

Peter Sels, June 20th, 2020. 
Copyright © 2020 Logically Yours BV.